In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import wandb

# 1. config
wandb.init(
    project='model',
    name='reg',
    config={
        'input_size': 20,
        'hidden_size1': 30,
        'hidden_size2': 10,
        'output_size': 1,

        'learning_rate': 0.001,
        'epoch_num': 5,
        'batch_size': 64
    }
)
config = wandb.config    

# 2. data
X = torch.randn(1000, config['input_size'])    
y = torch.randn(1000, config['output_size']) 

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

# 3. model
class MyModel(nn.Module): 
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) 
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))   
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MyModel(
    config['input_size'],
    config['hidden_size1'],
    config['hidden_size2'],
    config['output_size']
)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

# 4. train, eval

for epoch in range(config['epoch_num']):
    total_loss = 0
    for input, target in dataloader:
        # forward
        pred = model(input)
        loss = criterion(pred, target)
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    wandb.log({'epoch': epoch+1, 'loss': avg_loss})
    print(f'epoch:{epoch+1}, loss:{avg_loss:.4f}')